In [1]:
from to_objects import get_objects
from rdflib import Graph

CRITICAL:root:Install the 'bacnet-ingress' module, e.g. 'pip install buildingmotif[bacnet-ingress]'
DEBUG:buildingmotif.database.graph_connection:Creating tables for graph storage
DEBUG:buildingmotif.database.table_connection:Creating shape collection in model: 'urn:hpflex#'
DEBUG:buildingmotif.database.table_connection:Creating model: 'urn:hpflex#', with graph: 'ce3c5504-0361-40e0-85a0-f3e09685ee6a'
DEBUG:buildingmotif.database.graph_connection:Creating graph: 'ce3c5504-0361-40e0-85a0-f3e09685ee6a' in database with: 1 triples
DEBUG:buildingmotif.database.table_connection:Creating shape collection in library: 'brick-templates'
DEBUG:buildingmotif.database.table_connection:Creating database library: 'brick-templates'
DEBUG:buildingmotif.database.table_connection:Creating database template: 'hvac-zone'
DEBUG:buildingmotif.database.graph_connection:Creating graph: '7ae1cccb-df60-4aae-952c-56a79f39ea42' in database with: 1 triples
DEBUG:buildingmotif.database.table_connection:Creating data

In [2]:
g = Graph()
g.parse('tutorial/bldg2.ttl')

<Graph identifier=Nbd15bc9770924cbf84bc4f012cb9ef77 (<class 'rdflib.graph.Graph'>)>

In [3]:
zones = get_objects(g)

In [4]:
zone1 = zones[0]
zone1.spaces[0].area

Value(value=1500.0, unit='FT2')

In [5]:
zones[0].spaces

[Space(name='urn:hpflex/hpflex_demo#space_1_1', area=Value(value=1500.0, unit='FT2'))]

# Drafting

In [6]:
from buildingmotif import BuildingMOTIF
from buildingmotif.dataclasses import Library, Model
from rdflib import Graph, Literal, Namespace, URIRef
from semantic_mpc_interface.namespaces import * 
from semantic_mpc_interface.utils import * 
from buildingmotif.ingresses import CSVIngress, TemplateIngress, Record

In [7]:
bm = BuildingMOTIF("sqlite://")
model = Model.create(HPF)


DEBUG:buildingmotif.database.table_connection:Creating shape collection in model: 'urn:hpflex#'
DEBUG:buildingmotif.database.table_connection:Creating model: 'urn:hpflex#', with graph: '55c2e81a-9132-47c8-9284-fc197f11a2cf'
DEBUG:buildingmotif.database.graph_connection:Creating graph: '55c2e81a-9132-47c8-9284-fc197f11a2cf' in database with: 1 triples


In [8]:
tmp = Library.load(directory='src/semantic_mpc_interface/templates/brick-templates')

DEBUG:buildingmotif.database.table_connection:Creating database template: 'hvac-zone'
DEBUG:buildingmotif.database.graph_connection:Creating graph: '3c40cd09-7602-4dde-9bc6-72da4487cab4' in database with: 1 triples
DEBUG:buildingmotif.database.table_connection:Creating database template: 'space'
DEBUG:buildingmotif.database.graph_connection:Creating graph: '54f67557-7351-452f-8ebd-47aa313854b1' in database with: 2 triples
DEBUG:buildingmotif.database.table_connection:Creating database template: 'window'
DEBUG:buildingmotif.database.graph_connection:Creating graph: '6a5216f4-d554-4fe4-9267-b82a46a9b827' in database with: 4 triples
DEBUG:buildingmotif.database.table_connection:Creating database template: 'hp-rtu'
DEBUG:buildingmotif.database.graph_connection:Creating graph: 'b43a8571-6dc4-4078-a3bf-c713065f9d75' in database with: 6 triples
DEBUG:buildingmotif.database.table_connection:Creating database template: 'tstat'
DEBUG:buildingmotif.database.graph_connection:Creating graph: '72bd5

In [9]:
zone = tmp.get_template_by_name('hvac-zone')

In [10]:
zone_in = zone.inline_dependencies()

In [11]:
display(zone_in.all_parameters)

{'name',
 'space_name',
 'space_name-area_name',
 'space_name-area_unit',
 'space_name-area_value',
 'window_name',
 'window_name-area_name',
 'window_name-area_unit',
 'window_name-area_value',
 'window_name-azimuth_name',
 'window_name-azimuth_value',
 'window_name-tilt_name',
 'window_name-tilt_value'}

In [12]:
print(zone_in.body.serialize(format = 'ttl'))

@prefix brick: <https://brickschema.org/schema/Brick#> .
@prefix ns1: <http://qudt.org/schema/qudt/> .

<urn:___param___#name> a brick:HVAC_Zone ;
    brick:hasPart <urn:___param___#space_name>,
        <urn:___param___#window_name> .

<urn:___param___#space_name> a brick:Space ;
    brick:area <urn:___param___#space_name-area_name> .

<urn:___param___#space_name-area_name> ns1:hasQuantityKind <http://qudt.org/vocab/quantitykind/Area> ;
    ns1:hasUnit <urn:___param___#space_name-area_unit> ;
    brick:value <urn:___param___#space_name-area_value> .

<urn:___param___#window_name> a brick:Window ;
    brick:area <urn:___param___#window_name-area_name> ;
    brick:azimuth <urn:___param___#window_name-azimuth_name> ;
    brick:tilt <urn:___param___#window_name-tilt_name> .

<urn:___param___#window_name-area_name> ns1:hasQuantityKind <http://qudt.org/vocab/quantitykind/Area> ;
    ns1:hasUnit <urn:___param___#window_name-area_unit> ;
    brick:value <urn:___param___#window_name-area_value>

In [13]:
def get_var_name(graph, node):
    pre, ns, local = graph.compute_qname(node)
    if PARAM == ns:
        q_n = f"?{local}".replace('-','_')
    else:
        q_n = convert_to_prefixed(node, graph).replace('-','_')
    return q_n

In [14]:
def make_where(graph):
    where = []
    for s,p,o in graph.triples((None,None,None)):
        # print(s,p,o)
        qs = get_var_name(graph, s)
        qo = get_var_name(graph, o)
        qp = convert_to_prefixed(p, graph).replace('-','_')
        where.append(f"{qs} {qp} {qo} .")
    return "\n".join(where)

In [15]:
def get_query(graph):
     where = make_where(graph)
     prefixes = get_prefixes(graph)
     query = f"""{prefixes}\nSELECT DISTINCT * WHERE {{ {where} }}"""
     return query

In [16]:
query = get_query(zone_in.body)

In [17]:
print(query)

PREFIX brick: <https://brickschema.org/schema/Brick#>
PREFIX csvw: <http://www.w3.org/ns/csvw#>
PREFIX dc: <http://purl.org/dc/elements/1.1/>
PREFIX dcat: <http://www.w3.org/ns/dcat#>
PREFIX dcmitype: <http://purl.org/dc/dcmitype/>
PREFIX dcterms: <http://purl.org/dc/terms/>
PREFIX dcam: <http://purl.org/dc/dcam/>
PREFIX doap: <http://usefulinc.com/ns/doap#>
PREFIX foaf: <http://xmlns.com/foaf/0.1/>
PREFIX geo: <http://www.opengis.net/ont/geosparql#>
PREFIX odrl: <http://www.w3.org/ns/odrl/2/>
PREFIX org: <http://www.w3.org/ns/org#>
PREFIX prof: <http://www.w3.org/ns/dx/prof/>
PREFIX prov: <http://www.w3.org/ns/prov#>
PREFIX qb: <http://purl.org/linked-data/cube#>
PREFIX schema: <https://schema.org/>
PREFIX sh: <http://www.w3.org/ns/shacl#>
PREFIX skos: <http://www.w3.org/2004/02/skos/core#>
PREFIX sosa: <http://www.w3.org/ns/sosa/>
PREFIX ssn: <http://www.w3.org/ns/ssn/>
PREFIX time: <http://www.w3.org/2006/time#>
PREFIX vann: <http://purl.org/vocab/vann/>
PREFIX void: <http://rdfs.or

In [18]:
g = Graph()
g.parse('tutorial/bldg2.ttl')

<Graph identifier=N9df3c768776b45eeabf4508bc48554b3 (<class 'rdflib.graph.Graph'>)>

In [19]:
df = query_to_df(query, g)

In [20]:
class Value:
    unit: str
    value: float

In [21]:
df

,space_name_area_value,window_name_azimuth_value,space_name_area_name,window_name_tilt_value,space_name_area_unit,window_name_azimuth_name,window_name_area_name,window_name_area_unit,space_name,name,window_name_tilt_name,window_name_area_value,window_name
0,1500.0,180.0,urn:hpflex/hpflex_demo#space_1_1_area,90.0,unit:FT2,urn:hpflex/hpflex_demo#window_1_1_azimuth,urn:hpflex/hpflex_demo#window_1_1_area,unit:FT2,urn:hpflex/hpflex_demo#space_1_1,urn:hpflex/hpflex_demo#zone_1,urn:hpflex/hpflex_demo#window_1_1_tilt,150.0,urn:hpflex/hpflex_demo#window_1_1
1,1500.0,180.0,urn:hpflex/hpflex_demo#space_2_1_area,90.0,unit:FT2,urn:hpflex/hpflex_demo#window_2_1_azimuth,urn:hpflex/hpflex_demo#window_2_1_area,unit:FT2,urn:hpflex/hpflex_demo#space_2_1,urn:hpflex/hpflex_demo#zone_2,urn:hpflex/hpflex_demo#window_2_1_tilt,150.0,urn:hpflex/hpflex_demo#window_2_1
2,1500.0,180.0,urn:hpflex/hpflex_demo#space_3_1_area,90.0,unit:FT2,urn:hpflex/hpflex_demo#window_3_1_azimuth,urn:hpflex/hpflex_demo#window_3_1_area,unit:FT2,urn:hpflex/hpflex_demo#space_3_1,urn:hpflex/hpflex_demo#zone_3,urn:hpflex/hpflex_demo#window_3_1_tilt,150.0,urn:hpflex/hpflex_demo#window_3_1
3,1500.0,180.0,urn:hpflex/hpflex_demo#space_4_1_area,90.0,unit:FT2,urn:hpflex/hpflex_demo#window_4_1_azimuth,urn:hpflex/hpflex_demo#window_4_1_area,unit:FT2,urn:hpflex/hpflex_demo#space_4_1,urn:hpflex/hpflex_demo#zone_4,urn:hpflex/hpflex_demo#window_4_1_tilt,150.0,urn:hpflex/hpflex_demo#window_4_1


In [22]:
zone_in.generate_csv('test.csv')

In [23]:
zone_part = zone_in.evaluate({p:HPF[p] for p in zone_in.all_parameters if p.split('_')[-1] == 'name'})

/Users/lazlopaul/Desktop/HPflex/Semantic_MPC_Interface/.venv/lib/python3.10/site-packages/buildingmotif/dataclasses/template.py:420: UserWarning: Parameters "window_name-area_unit, window_name-tilt_value, space_name-area_value, window_name-azimuth_value, space_name-area_unit, window_name-area_value" were not provided during evaluation
  warnings.warn(


In [24]:
zone_part.generate_csv('test.csv')
# dependencies are showing up in headers for some reason - can just remove these. 
# dependencies = zone_part.get_dependencies()

# df = pd.read_csv('test.csv')

# remove_cols = [dependency.template.name for dependency in zone.get_dependencies()]
# df = df.drop(columns=remove_cols)

# # Save the new CSV
# df.to_csv('test.csv', index=False)


In [25]:
site = tmp.get_template_by_name('site')
site.inline_dependencies().generate_csv('test.csv')

In [1]:
from semantic_mpc_interface import HPFlexSurvey

CRITICAL:root:Install the 'bacnet-ingress' module, e.g. 'pip install buildingmotif[bacnet-ingress]'


In [2]:
s = HPFlexSurvey('test_site','test_build','.', overwrite=True)

DEBUG:buildingmotif.database.graph_connection:Creating tables for graph storage
DEBUG:buildingmotif.database.table_connection:Creating shape collection in library: 'brick-templates'
DEBUG:buildingmotif.database.table_connection:Creating database library: 'brick-templates'
DEBUG:buildingmotif.database.table_connection:Creating database template: 'hvac-zone'
DEBUG:buildingmotif.database.graph_connection:Creating graph: '759b071e-19cb-4ccc-9709-89b23b1eb2af' in database with: 1 triples
DEBUG:buildingmotif.database.table_connection:Creating database template: 'space'
DEBUG:buildingmotif.database.graph_connection:Creating graph: '5cea8aec-856c-41ff-b28d-91d98b6476a1' in database with: 2 triples
DEBUG:buildingmotif.database.table_connection:Creating database template: 'window'
DEBUG:buildingmotif.database.graph_connection:Creating graph: '86670343-5f30-4bb3-a3e3-0635d1abf14a' in database with: 4 triples
DEBUG:buildingmotif.database.table_connection:Creating database template: 'hp-rtu'
DEBUG:

{'area_unit', 'name', 'area_name', 'area_value'}
{'heating_COP_name', 'heating_capacity_name', 'cooling_COP_value', 'cooling_capacity_value', 'heating_COP_value', 'cooling_capacity_name', 'name', 'heating_capacity_value', 'cooling_COP_name'}
{'setpoint_deadband-unit', 'tolerance-value', 'active-value', 'resolution-value', 'resolution', 'stage_count', 'stage_count-value', 'setpoint_deadband-value', 'setpoint_deadband', 'tolerance', 'active', 'name', 'tolerance-unit', 'resolution-unit'}
{'azimuth_name', 'area_name', 'area_unit', 'area_value', 'tilt_name', 'name', 'azimuth_value', 'tilt_value'}


In [3]:
s.easy_config(zone_space_window_list=[(2,2),(1,2),(1,3)])

{'site_id': 'test_site', 'hvac_type': 'hp-rtu', 'empty_params': {'space': ['area_name'], 'hp-rtu': ['heating_COP_name', 'heating_capacity_name', 'cooling_capacity_name', 'cooling_COP_name'], 'tstat': [], 'window': ['azimuth_name', 'area_name', 'tilt_name']}, 'param_mapping': {'space': {'area_value': 'area'}, 'hp-rtu': {'cooling_COP_value': 'cooling_COP', 'cooling_capacity_value': 'cooling_capacity', 'heating_COP_value': 'heating_COP', 'heating_capacity_value': 'heating_capacity'}, 'tstat': {}, 'window': {'area_value': 'area', 'azimuth_value': 'azimuth', 'tilt_value': 'tilt'}}, 'hvacs_feed_hvacs': {}, 'hvacs_feed_zones': {'hvac_1': ['zone_1'], 'hvac_2': ['zone_2'], 'hvac_3': ['zone_3']}, 'zones_contain_spaces': {'zone_1': ['space_1_1', 'space_1_2'], 'zone_2': ['space_2_1'], 'zone_3': ['space_3_1']}, 'zones_contain_windows': {'zone_1': ['window_1_1', 'window_1_2'], 'zone_2': ['window_2_1', 'window_2_2'], 'zone_3': ['window_3_1', 'window_3_2', 'window_3_3']}}
